# FIT5148 Assessment 1
#### Student Name: John Paul Mariano
#### Student ID: 28037537

Date: 12/09/2017
Version: 1.0
Environment: Python 3 and Jupyter notebook

Libraries used:
* cx_oracle: to work with Oracle
* pandas:for dataframe

# Table of Contents
1. Database<br/>
  1.1. Inroduction<br/>
  1.2. Database Objects Creation<br/>
  1.3. ER Diagram<br/>
  1.4. Indexing<br/>
  1.5. Population<br/>
2. Oracle Stored Procedure<br/>
3. Triggers<br/>
4. Fragmentation<br/>
5. ETL Demonstration<br/>

# Introduction
Maximum one page that briefly lists which tasks you have completed and any additional functionality or features that you have done.

# Task 1- Database (max 8 marks)
##  1.1 Introduction 
Explain attribute names and data types, and relationships between tables. You need to make decisions on using appropriate **PK**s and **FK**s. You will add integrity constraints, check constraints or triggers to achieve the requirements and conditions.

## 1.1.1 Establish Connections

In [1]:
# import required libraries
import cx_Oracle
import pandas as pd

In [2]:
# create function to connect to database
def connect(db_name, user_id, password):
    dsn = cx_Oracle.makedsn(host='hippo.its.monash.edu', port='1521', sid=db_name) # pass data source name
    conn = cx_Oracle.connect(user=user_id, password=password, dsn=dsn) # connect to database
    cur = conn.cursor() # cursor() methods opens a cursor for statements to use. 
    return cur, conn #return cursor and connection

In [3]:
# declare variables for credentials
my_id = 'S28037537'
my_pass = 'student'

Now, we connect to **FIT5148A** and **FIT5148B**:

In [4]:
# connect to site A
cur_a, conn_a = connect('FIT5148A', my_id, my_pass)

In [5]:
# connect to site B
cur_b, conn_b = connect('FIT5148B', my_id, my_pass)

## 1.1.2 Preparation
To enable running of the codes in this notebook multiple times without issues, a cleanup of the database objects (if they exist) is done at this point.

The following are helper functions used for database objects management:

In [6]:
# function to drop a table
# exception when table does not exist is ignored
def drop_table(cur, table_name):
    sql = '''
        BEGIN
            EXECUTE IMMEDIATE 'DROP TABLE {}';
        EXCEPTION
            WHEN OTHERS THEN
                IF SQLCODE != -942 THEN
                    RAISE;
                END IF;
        END;'''.format(table_name)
    cur.execute(sql)

In [7]:
def drop_constraint(cur, table_name, constraint_name):
    sql = '''
        BEGIN
            EXECUTE IMMEDIATE 'ALTER TABLE {} DROP CONSTRAINT {}';
        EXCEPTION
            WHEN OTHERS THEN
                IF (SQLCODE != -2443 AND SQLCODE = 942) THEN
                    RAISE;
                END IF;
        END;'''.format(table_name, constraint_name)
    cur.execute(sql)

In [8]:
def drop_sequence(cur, sequence_name):
    sql = '''
        BEGIN
            EXECUTE IMMEDIATE 'DROP SEQUENCE {}';
        EXCEPTION
            WHEN OTHERS THEN
                IF SQLCODE != -2289 THEN
                    RAISE;
                END IF;
        END;'''.format(sequence_name)
    cur.execute(sql)

In [9]:
def drop_index(cur, index_name):
    sql = '''
        BEGIN
            EXECUTE IMMEDIATE 'DROP INDEX {}';
        EXCEPTION
            WHEN OTHERS THEN
                IF SQLCODE != -1418 THEN
                    RAISE;
                END IF;
        END;'''.format(index_name)
    cur.execute(sql)

First, cleanup objects (if existing) in **FIT5148A**

In [10]:
# Drop tables
drop_table(cur_a, 'hotel')

In [11]:
# commit changes
conn_a.commit()

Next, cleanup objects (if existing) in **FIT5148B**

In [12]:
# Drop constraints
# Need to drop foreign key constraints before dropping tables
drop_constraint(cur_b, 'customer', 'customer_membership_fk')
drop_constraint(cur_b, 'room', 'room_room_type_fk')
drop_constraint(cur_b, 'booking', 'booking_customer_fk')
drop_constraint(cur_b, 'booked_room', 'booked_room_booking_fk')
drop_constraint(cur_b, 'booked_room', 'booked_room_room_fk')
drop_constraint(cur_b, 'payment', 'payment_booking_fk')

In [13]:
# Drop indexes
drop_index(cur_b, 'booked_room_dates_idx')
drop_index(cur_b, 'room_attrib_idx')

In [14]:
# Drop sequences
drop_sequence(cur_b, 'payment_seq')

In [15]:
# Drop tables
drop_table(cur_b, 'room')
drop_table(cur_b, 'room_type')
drop_table(cur_b, 'customer')
drop_table(cur_b, 'membership')
drop_table(cur_b, 'booking')
drop_table(cur_b, 'booked_room')
drop_table(cur_b, 'payment')

In [16]:
# commit changes
conn_b.commit()

## 1.2 Database Objects Creation
Database tables will be distributed between two locations, **FIT5148A** and **FIT5148B**.<br/>
<br/>
The **Hotel** table will be created in **FIT5148A** while all the rest are in **FIT5148B**.<br/>
<br/>
First, create the table in site **FIT5148A**.

### 1.2.1 Tables
### Hotel table
The Hotel table stores information about hotels.<br/>
The hotel_id column is set as the primary key.

In [17]:
cur_a.execute('''
CREATE TABLE hotel (
    hotel_id NUMBER(10) NOT NULL,
    hotel_name VARCHAR2(40) NOT NULL,
    hotel_type VARCHAR2(10),
    construct_year NUMBER(4),
    country VARCHAR2(30),
    city VARCHAR2(30),
    address VARCHAR2(100),
    contact_num VARCHAR2(30),
    contact_email VARCHAR2(40),
    CONSTRAINT hotel_pk PRIMARY KEY (hotel_id)
)''')

In [18]:
# commit changes
conn_a.commit()

Next, create the tables in site **FIT5148B**

### Room table
The Room table stores data about hotel rooms.<br/>
The primary key is a compound key consisting of the hotel_id and room_num.

In [19]:
cur_b.execute('''
CREATE TABLE room (
    hotel_id NUMBER(10) NOT NULL,
    room_num NUMBER(6) NOT NULL,
    room_type_code VARCHAR2(3),
    occupancy NUMBER(2),
    rate NUMBER(5,2),
    description VARCHAR2(60),
    CONSTRAINT room_pk PRIMARY KEY (hotel_id, room_num)
)''')

### Room_Type table
The Room_Type table contains details about different room types.<br/>
The room_type_code is the primary key, and is referenced by the Room table.

In [20]:
cur_b.execute('''
CREATE TABLE room_type (
    room_type_code VARCHAR2(3) NOT NULL,
    room_type_name VARCHAR2(20) NOT NULL,
    CONSTRAINT room_type_pk PRIMARY KEY (room_type_code)
)''')

### Customer table
The Customer table stores details about the customers who book for hotel rooms.<br/>
The primary key is the cust_id.

In [21]:
cur_b.execute('''
CREATE TABLE customer (
    cust_id NUMBER(10) NOT NULL,
    first_name VARCHAR2(30) NOT NULL,
    last_name VARCHAR2(30) NOT NULL,
    title VARCHAR2(5),
    member_tier_code VARCHAR2(3),
    member_credit NUMBER(10),
    contact_num VARCHAR2(30),
    contact_email VARCHAR2(40),
    CONSTRAINT customer_pk PRIMARY KEY (cust_id)
)''')

### Membership table
The Membership table stores details about the membership tier levels.<br/>
The member_tier_code is the primary key; and it is referenced by the Customer table.

In [22]:
cur_b.execute('''
CREATE TABLE membership (
    member_tier_code VARCHAR2(3) NOT NULL,
    member_tier_name VARCHAR2(30) NOT NULL,
    tier_credit NUMBER(6),
    discount NUMBER(5,2),
    other_reward VARCHAR2(50),
    CONSTRAINT membership_pk PRIMARY KEY (member_tier_code)
)''')

### Booking table
The Booking table stores information about bookings made by customers.<br/>
The primary key is a booking_id.<br/>
Each booking entry can have one or more rooms so they are stored in a separate table, Booked_Room<br/>
The Booking table also references the Customer table

In [23]:
cur_b.execute('''
CREATE TABLE booking (
    booking_id NUMBER(10) NOT NULL,
    cust_id NUMBER(10) NOT NULL,
    contact_name VARCHAR2(30),
    contact_num VARCHAR2(30),
    contact_email VARCHAR2(40),
    total_amount NUMBER(12,2),
    payment_status VARCHAR2(10),
    CONSTRAINT booking_pk PRIMARY KEY (booking_id)
)''')

### Booked_Room
The Booked_Room table contains records for the booked rooms.<br/>
Each record in the Booking table can have one or more related records in the Booked_Room table.<br/>
The primary key is a compound key.
This table references the Booking table as well as the Room table. 

In [24]:
cur_b.execute('''
CREATE TABLE booked_room (
    booking_id NUMBER(10) NOT NULL,
    hotel_id NUMBER(10) NOT NULL,
    room_num NUMBER(6) NOT NULL,
    guest_num NUMBER(2),
    checkin_date DATE NOT NULL,
    checkout_date DATE NOT NULL,
    CONSTRAINT booked_room_pk PRIMARY KEY (booking_id, hotel_id, room_num, checkin_date, checkout_date)
)''')

### Payment  table
The Payment table contains records of payments.<br/>
The primary key is the payment_id column and it is automatically generated using a sequence.<br/>
It references the Booking table using the booking_id column.

In [25]:
cur_b.execute('''
CREATE TABLE payment (
    payment_id NUMBER(10) NOT NULL,
    booking_id NUMBER(10) NOT NULL,
    payment_date DATE,
    payment_method VARCHAR2(20),
    payment_amount NUMBER(12,2),
    CONSTRAINT payment_pk PRIMARY KEY (payment_id)
)''')

In [26]:
# commit changes
conn_b.commit()

### 1.2.2 Create Foreign Keys
To establish relationships between tables in **FIT5148B**, foreign keys will be created.<br/>
<br/>
Note that for the relationship and integrity between the **Hotel** table in **FIT5148A** and the related **Room** table in **FIT5148B**, triggers will be used and will be illustrated in a succeeding section below.<br/>
This is because the two tables are in different locations.

In [27]:
# the customer table references the membership table for the member_tier_code values
cur_b.execute('''
ALTER TABLE customer
    ADD CONSTRAINT customer_membership_fk
        FOREIGN KEY (member_tier_code)
        REFERENCES membership(member_tier_code)
''')

In [28]:
# the room table references the room_type table for the room_type_code values
cur_b.execute('''
ALTER TABLE room
    ADD CONSTRAINT room_room_type_fk
        FOREIGN KEY (room_type_code)
        REFERENCES room_type(room_type_code)
''')

In [29]:
# the booking table references the customer table for the cust_id customer identifier values
cur_b.execute('''
ALTER TABLE booking
    ADD CONSTRAINT booking_customer_fk
        FOREIGN KEY (cust_id)
        REFERENCES customer(cust_id)
''')

In [30]:
# the booked_room table references the booking table for the booking_id identifier values
cur_b.execute('''
ALTER TABLE booked_room
    ADD CONSTRAINT booked_room_booking_fk
        FOREIGN KEY (booking_id)
        REFERENCES booking(booking_id)
''')

In [31]:
# the booked_room table references the room table for the (hotel_id and room_num) identifier values
cur_b.execute('''
ALTER TABLE booked_room
    ADD CONSTRAINT booked_room_room_fk
        FOREIGN KEY (hotel_id, room_num)
        REFERENCES room(hotel_id, room_num)
''')

In [32]:
# the payment table references the booking table for the booking_id identifier values
cur_b.execute('''
ALTER TABLE payment
    ADD CONSTRAINT payment_booking_fk
        FOREIGN KEY (booking_id)
        REFERENCES booking(booking_id)
''')

In [33]:
# commit changes
conn_b.commit()

### 1.2.3 Triggers
Triggers will be added to specific tables to implement some rules regarding bookings:<br/>
- The number of guests in a booked room cannot exceed room occupancy.

In [34]:
# trigger before an insert or update to the booked_room table
# it should check that the number of guests does not exceed room occupancy
cur_b.execute('''
CREATE OR REPLACE TRIGGER booked_room_trigger_biu
BEFORE INSERT OR UPDATE ON booked_room
FOR EACH ROW
DECLARE 
    maxOccupancy INTEGER;
BEGIN
    -- get max occupancy
    SELECT occupancy INTO maxOccupancy FROM room WHERE hotel_id=:new.hotel_id AND room_num=:new.room_num;
    -- raise an error if number of guests is greater than room occupancy
    IF (:new.guest_num > maxOccupancy) THEN
        RAISE_APPLICATION_ERROR(-20000, 'Number of guests is greater than room occupancy');
    END IF;    
END;
''')
conn_b.commit()

- Total amount for booking will be computed automatically taking into account the membership discount.

In [35]:
# We will create a view to help in accomplishing this
# View for the customer discounts
cur_b.execute('''
CREATE OR REPLACE VIEW customer_discounts AS
    SELECT c.cust_id, m.discount 
    FROM customer c INNER JOIN MEMBERSHIP m ON c.member_tier_code=m.member_tier_code''')
conn_b.commit()

In [36]:
# trigger after an insert or update to the booked_room table
# it will compute the total amount due and update the booking table
# it also takes into account the membership discount
cur_b.execute('''
CREATE OR REPLACE TRIGGER booked_room_trigger_aiu
AFTER INSERT OR UPDATE ON booked_room
FOR EACH ROW
DECLARE 
    baseRate NUMBER;
    discount NUMBER;
    discountedAmount NUMBER;
BEGIN
    -- get rate for booked room
    SELECT rate INTO baseRate FROM room WHERE hotel_id=:new.hotel_id AND room_num=:new.room_num;
    -- get discount rate for the customer
    SELECT discount INTO discount FROM customer_discounts WHERE cust_id= 
        (SELECT cust_id FROM booking where booking_id=:new.booking_id);
    -- compute amount
    discountedAmount := baseRate * (100 - discount) / 100;
    -- update booking table total amount
    UPDATE booking SET total_amount = (total_amount + discountedAmount) WHERE booking_id=:new.booking_id;
END;
''')
conn_b.commit()

- Payment amount should be added to the membership credits of the customer
- Payment should update the payment status to 'Received'

In [37]:
# trigger after an insert to the payment table
# it will do the following:
# - update the membership credits of the customer
# - set the payment status of the booking to 'Received' (if payment)
#   or set it to 'Cancelled' (if reverse is done, i.e. negative amount)
cur_b.execute('''
CREATE OR REPLACE TRIGGER payment_trigger_aiu
AFTER INSERT OR UPDATE ON payment
FOR EACH ROW
BEGIN
    IF (:new.payment_amount > 0) THEN
        -- update payment status
        UPDATE booking SET payment_status = 'Received' WHERE booking_id = :new.booking_id;
    ELSIF (:new.payment_amount < 0) THEN
        -- update payment status
        UPDATE booking SET payment_status = 'Cancelled' WHERE booking_id = :new.booking_id;
    END IF;
    -- update customer credits
    UPDATE customer SET member_credit = (member_credit + :new.payment_amount) WHERE cust_id =
        (SELECT cust_id FROM booking WHERE booking_id = :new.booking_id);
END;
''')
conn_b.commit()

## 1.3 ER Diagram
![ER Diagram](./hotel_booking_system_erd.png)

#### ER Diagram details:
The **Hotel** table is in **FIT548A** while all the rest are in **FIT5148B**.<br/>
<br/>
Each **Hotel** record can have one or more related records in the **Room** table. But each room can only be related to one hotel.<br/>
<br/>
Each **Room** record can have 0 or more **Booked_Room** related records. Each booked room can only be associated to one and only one room.<br/>
<br/>
Each **Room** entry should refer to exactly one **Room_Type**.<br/>
<br/>
Each **Booking** record can have one or more **Booked_Room** related records. Each booked room can only be associated to one booking record.<br/>
<br/>
Each **Booking** is associated with one **Customer** record. But each customer record can have 0 or multiple bookings.<br/>
<br/>
Each **Customer** record has exactly one **Membership** type. The membership types can be referenced by 0 or more customer records.<br/>
<br/>
Each **Booking** has one or more related **Payment** records. Each payment is associated to one booking record.

## 1.4 Indexing
Create at least one index for an appropriate attribute and sequences where necessary and increment them by 1.

#### Booking Table Index
We will create an index in the **Booking** table for the **check-in** and **check-out** columns because queries against these columns are likely to be made when searching for available rooms.<br/>

In [38]:
cur_b.execute('''
CREATE INDEX booked_room_dates_idx
    ON booked_room (checkin_date, checkout_date)
''')

#### Room Table Index
Indexes on the Room table are also added on the columns that will be likely used as criteria by users when searching for rooms, such as the **room type**, **occupancy/capacity** and **room rate**.

In [39]:
cur_b.execute('''
CREATE INDEX room_attrib_idx
    ON room (room_type_code, occupancy, rate)
''')

In [40]:
conn_b.commit()

### Sequence

#### Payment Sequence
We can use a sequence as unique identifier for each booking transaction. It will be set using a trigger on the **Payment** table each time a new payment record is added.

In [41]:
# create sequence
cur_b.execute('''
CREATE SEQUENCE payment_seq
    START WITH     1
    INCREMENT BY   1
    NOCACHE
    NOCYCLE    
''')

In [42]:
# create trigger when record is inserted into table
# set the payment_id to the next sequence value
cur_b.execute('''
CREATE OR REPLACE TRIGGER payment_trigger_bir
    BEFORE INSERT ON payment
    FOR EACH ROW
    BEGIN
        SELECT payment_seq.NEXTVAL
        INTO :new.payment_id
        FROM DUAL;
    END;
''')

In [43]:
conn_b.commit()

## 1.5 Population 
Populate your tables with meaningful data. You will use this data to show the results of queries in Task 5. Therefore, make sure your data covers different scenarios so you can prove that your design work perfect in different scenarios (you may lose points otherwise).

In [44]:
# function to populate data into a table
def insert_rows(cur, conn, table_name, data, col_names=None, date_format=None):
    try:
        col_count = len(data[0]) # column/values count 
        # set right number of column parameters for the table.
        # it is based on the number of items in each tuple in the data list
        col_params = ''
        for i in range(1, col_count + 1):
            col_params += ':{}'.format(i)
            if (i < col_count):
                col_params += ', '
                
        # if date format passed, set session date format
        if (date_format):
            cur.execute('ALTER SESSION SET NLS_DATE_FORMAT=\'' + date_format + '\'')

        # prepare statement for inserting data
        if (col_names):
            sql = 'insert into ' + table_name + '(' + col_names + ') values (' + col_params + ')'
        else:
            sql = 'insert into ' + table_name + ' values (' + col_params + ')'
        cur.prepare(sql)
        
        # execute the sql insert for all rows and commit
        cur.executemany(None, data)
        conn.commit()
        
    except Exception as ex:
        print('Exception while inserting rows', ex)

#### Hotel table data

In [45]:
# declare list of tuples for the data
hotel_data = [
    (1, 'ParkRoyal Darling Harbour', '4 Star', 1999, 'Australia', 'Sydney', '161 Elizabeth Street, Sydney, NSW 2000', '02-9286-6000', 'reservation@sheratonsyd.com'),
    (2, 'Hyatt Regency Sydney', '5 Star', 1995, 'Australia', 'Sydney', '161 Sussex Street, Sydney, NSW 2000', '02-8069-5434', 'reservation@hyattsyd.com'),
    (3, 'InterContinental Rialto', '5 Star', 2001, 'Australia', 'Melbourne', '495 Collins Street, Melbourne, VIC 3000' , '1800-760595', 'reservation@interconmelb.com'),
    (4, 'Vibe Savoy Hotel', '3 Star', 2010, 'Australia', 'Melbourne', '630 Little Collins St, Melbourne, VIC 3000', '03-8797-5228', 'reservation@vibesavoy.com'),
    (5, 'Hilton Brisbane', '4 Star', 2002, 'Australia', 'Brisbane', '190 Elizabeth St, Brisbane, QLD 4000', '07-3234-2000', 'reservation@hiltonqld.com'),
    (6, 'Novotel Surfers Paradise', '4 Star', 2012, 'Australia' , 'Gold Coast', 'Cnr Surfers Paradise Blvd Hanlan Street, QLD 4217', '07-1234-2000', 'reservation@novotel.com')
]

# insert data into table
insert_rows(cur_a, conn_a, 'hotel', hotel_data)

In [46]:
# check inserted records
df = pd.read_sql('SELECT * from hotel', con=conn_a)
df

,HOTEL_ID,HOTEL_NAME,HOTEL_TYPE,CONSTRUCT_YEAR,COUNTRY,CITY,ADDRESS,CONTACT_NUM,CONTACT_EMAIL
0,1,ParkRoyal Darling Harbour,4 Star,1999,Australia,Sydney,"161 Elizabeth Street, Sydney, NSW 2000",02-9286-6000,reservation@sheratonsyd.com
1,2,Hyatt Regency Sydney,5 Star,1995,Australia,Sydney,"161 Sussex Street, Sydney, NSW 2000",02-8069-5434,reservation@hyattsyd.com
2,3,InterContinental Rialto,5 Star,2001,Australia,Melbourne,"495 Collins Street, Melbourne, VIC 3000",1800-760595,reservation@interconmelb.com
3,4,Vibe Savoy Hotel,3 Star,2010,Australia,Melbourne,"630 Little Collins St, Melbourne, VIC 3000",03-8797-5228,reservation@vibesavoy.com
4,5,Hilton Brisbane,4 Star,2002,Australia,Brisbane,"190 Elizabeth St, Brisbane, QLD 4000",07-3234-2000,reservation@hiltonqld.com
5,6,Novotel Surfers Paradise,4 Star,2012,Australia,Gold Coast,"Cnr Surfers Paradise Blvd Hanlan Street, QLD 4217",07-1234-2000,reservation@novotel.com


#### Room_Type table data

In [47]:
# declare list of tuples for the data
room_type_data = [
    ('SGL', 'Single room'),
    ('DBL', 'Double room'),
    ('TWN', 'Twin room'),
    ('Q', 'Queen room'),
    ('K', 'King room'),
    ('STU', 'Studio room')
]

# insert data into table
insert_rows(cur_b, conn_b, 'room_type', room_type_data)

In [48]:
# check inserted records
df = pd.read_sql('SELECT * from room_type', con=conn_b)
df

,ROOM_TYPE_CODE,ROOM_TYPE_NAME
0,SGL,Single room
1,DBL,Double room
2,TWN,Twin room
3,Q,Queen room
4,K,King room
5,STU,Studio room


#### Room table data

In [49]:
# declare list of tuples for the data
room_data = [
    (1, 101, 'K', 3, 230.00, 'King room, smoking'),
    (1, 102, 'K', 3, 230.00, 'King room, smoking'),
    (1, 103, 'K', 3, 230.00, 'King room, smoking'),
    (1, 104, 'K', 3, 230.00, 'King room, smoking'),
    (1, 105, 'K', 3, 230.00, 'King room, smoking'),
    (1, 106, 'K', 3, 230.00, 'King room, smoking'),
    (1, 201, 'STU', 2, 310.00, 'Studio suite, non-smoking'),
    (1, 202, 'STU', 2, 310.00, 'Studio suite, non-smoking'),
    (1, 203, 'STU', 2, 310.00, 'Studio suite, non-smoking'),
    (1, 204, 'STU', 2, 310.00, 'Studio suite, non-smoking'),
    (1, 205, 'STU', 2, 310.00, 'Studio suite, non-smoking'),
    (1, 206, 'STU', 2, 310.00, 'Studio suite, non-smoking'),
    (2, 101, 'TWN', 2, 190.00, 'Twin room, non-smoking'),
    (2, 102, 'TWN', 2, 190.00, 'Twin room, non-smoking'),
    (2, 103, 'TWN', 2, 190.00, 'Twin room, non-smoking'),
    (2, 104, 'TWN', 2, 190.00, 'Twin room, non-smoking'),
    (2, 105, 'TWN', 2, 190.00, 'Twin room, non-smoking'),
    (2, 106, 'TWN', 2, 190.00, 'Twin room, non-smoking'),
    (2, 201, 'SGL', 1, 140.00, 'Single room, smoking'),
    (2, 202, 'SGL', 1, 140.00, 'Single room, smoking'),
    (2, 203, 'SGL', 1, 140.00, 'Single room, smoking'),
    (2, 204, 'SGL', 1, 140.00, 'Single room, smoking'),
    (2, 205, 'SGL', 1, 140.00, 'Single room, smoking'),
    (2, 206, 'SGL', 1, 140.00, 'Single room, smoking'),
    (3, 101, 'Q', 2, 185.50, 'Twin room, smoking'),
    (3, 102, 'Q', 2, 185.50, 'Twin room, smoking'),
    (3, 103, 'Q', 2, 185.50, 'Twin room, smoking'),
    (3, 104, 'Q', 2, 185.50, 'Twin room, smoking'),
    (3, 105, 'Q', 2, 185.50, 'Twin room, smoking'),
    (3, 106, 'Q', 2, 185.50, 'Twin room, smoking'),
    (3, 201, 'K', 2, 285.50, 'King room, smoking'),
    (3, 202, 'K', 2, 285.50, 'King room, smoking'),
    (3, 203, 'K', 2, 285.50, 'King room, smoking'),
    (3, 204, 'K', 2, 285.50, 'King room, smoking'),
    (3, 205, 'K', 2, 285.50, 'King room, smoking'),
    (3, 206, 'K', 2, 285.50, 'King room, smoking'),
    (4, 101, 'TWN', 2, 155.00, 'Twin room, non-smoking'),
    (4, 102, 'TWN', 2, 155.00, 'Twin room, non-smoking'),
    (4, 103, 'TWN', 2, 155.00, 'Twin room, non-smoking'),
    (4, 104, 'TWN', 2, 155.00, 'Twin room, non-smoking'),
    (4, 105, 'TWN', 2, 155.00, 'Twin room, non-smoking'),
    (4, 106, 'TWN', 2, 155.00, 'Twin room, non-smoking'),
    (4, 201, 'STU', 2, 295.00, 'Studio suite, non-smoking'),
    (4, 202, 'STU', 2, 295.00, 'Studio suite, non-smoking'),
    (4, 203, 'STU', 2, 295.00, 'Studio suite, non-smoking'),
    (4, 204, 'STU', 2, 295.00, 'Studio suite, non-smoking'),
    (4, 205, 'STU', 2, 295.00, 'Studio suite, non-smoking'),
    (4, 206, 'STU', 2, 295.00, 'Studio suite, non-smoking'),
    (5, 101, 'SGL', 1, 100.50, 'Single room, non-smoking'),
    (5, 102, 'SGL', 1, 100.50, 'Single room, non-smoking'),
    (5, 103, 'SGL', 1, 100.50, 'Single room, non-smoking'),
    (5, 104, 'SGL', 1, 100.50, 'Single room, non-smoking'),
    (5, 105, 'SGL', 1, 100.50, 'Single room, non-smoking'),
    (5, 106, 'SGL', 1, 100.50, 'Single room, non-smoking'),
    (5, 201, 'DBL', 2, 300.00, 'King room, non-smoking'),
    (5, 202, 'DBL', 2, 300.00, 'King room, non-smoking'),
    (5, 203, 'DBL', 2, 300.00, 'King room, non-smoking'),
    (5, 204, 'DBL', 2, 300.00, 'King room, non-smoking'),
    (5, 205, 'DBL', 2, 300.00, 'King room, non-smoking'),
    (5, 206, 'DBL', 2, 300.00, 'King room, non-smoking'),
    (6, 101, 'TWN', 2, 280.50, 'Twin room, smoking'),
    (6, 102, 'TWN', 2, 280.50, 'Twin room, smoking'),
    (6, 103, 'TWN', 2, 280.50, 'Twin room, smoking'),
    (6, 104, 'TWN', 2, 280.50, 'Twin room, smoking'),
    (6, 105, 'TWN', 2, 280.50, 'Twin room, smoking'),
    (6, 106, 'TWN', 2, 280.50, 'Twin room, smoking'),
    (6, 201, 'STU', 2, 390.50, 'Studio Suite, smoking'),
    (6, 202, 'STU', 2, 390.50, 'Studio Suite, smoking'),
    (6, 203, 'STU', 2, 390.50, 'Studio Suite, smoking'),
    (6, 204, 'STU', 2, 390.50, 'Studio Suite, smoking'),
    (6, 205, 'STU', 2, 390.50, 'Studio Suite, smoking'),
    (6, 206, 'STU', 2, 390.50, 'Studio Suite, smoking')
]

# insert data into table
insert_rows(cur_b, conn_b, 'room', room_data)

In [50]:
# check inserted records
df = pd.read_sql('SELECT * from room', con=conn_b)
df.head(10) # show 10 rows only

,HOTEL_ID,ROOM_NUM,ROOM_TYPE_CODE,OCCUPANCY,RATE,DESCRIPTION
0,1,101,K,3,230.0,"King room, smoking"
1,1,102,K,3,230.0,"King room, smoking"
2,1,103,K,3,230.0,"King room, smoking"
3,1,104,K,3,230.0,"King room, smoking"
4,1,105,K,3,230.0,"King room, smoking"
5,1,106,K,3,230.0,"King room, smoking"
6,1,201,STU,2,310.0,"Studio suite, non-smoking"
7,1,202,STU,2,310.0,"Studio suite, non-smoking"
8,1,203,STU,2,310.0,"Studio suite, non-smoking"
9,1,204,STU,2,310.0,"Studio suite, non-smoking"


#### Membership table data

In [51]:
# declare list of tuples for the data
membership_data = [
    ('RED', 'Red Standard', 2000, 0.5, 'Welcome drinks'),
    ('BRO', 'Bronze Elite', 5000, 2, 'Dinner for 2'),
    ('SIL', 'Silver Elite', 10000, 5, 'Dinner for 2, Theatre tickets'),
    ('GOL', 'Gold Elite', 25000, 8, 'Butler service'),
    ('PLA', 'Platinum Elite', 50000, 12, 'Butler service, valet, tickets')
]

# insert data into table
insert_rows(cur_b, conn_b, 'membership', membership_data)

In [52]:
# check inserted records
df = pd.read_sql('SELECT * from membership', con=conn_b)
df

,MEMBER_TIER_CODE,MEMBER_TIER_NAME,TIER_CREDIT,DISCOUNT,OTHER_REWARD
0,RED,Red Standard,2000,0.5,Welcome drinks
1,BRO,Bronze Elite,5000,2.0,Dinner for 2
2,SIL,Silver Elite,10000,5.0,"Dinner for 2, Theatre tickets"
3,GOL,Gold Elite,25000,8.0,Butler service
4,PLA,Platinum Elite,50000,12.0,"Butler service, valet, tickets"


#### Customer table data

In [53]:
# declare list of tuples for the data
customer_data = [
    (1, 'Malcolm', 'Leja', 'Mr', 'SIL', 11000, '03-2477-9133', 'malcom@leja.com.au'),
    (2, 'Claudia', 'Gawrych', 'Ms', 'BRO', 6500, '02-4246-3092', 'claudia@gmail.com'),
    (3, 'Eliseo', 'Mikovec', 'Mr', 'RED', 3800, '02-9829-2371', 'emikovec@mikovec.com.au'),
    (4, 'Colene', 'Tolbent', 'Ms', 'GOL', 25500, '02-4376-1104', 'colene.tolbent@tolbent.net.au'),
    (5, 'Chester', 'Dollins', 'Mr', 'PLA', 80000, '02-1622-6412', 'chester_dollins@gmail.com'),
    (6, 'Armando', 'Barkley', 'Mr', 'GOL', 48000, '07-8371-4719', 'armando.barkley@yahoo.com'),
    (7, 'Tamie', 'Hollimon', 'Ms', 'BRO', 7800, '03-2444-8291', 'tamie@hollimon.com.au'),
    (8, 'Dylan', 'Chaleun', 'Mr', 'SIL', 15500, '08-7915-5110', 'dylan_chaleun@hotmail.com'),
    (9, 'John', 'Smith', 'Mr', 'SIL', 15500, '08-7915-8889', 'john.smith@hotmail.com'),
    (10, 'Jon', 'Smith', 'Mr', 'PLA', 65000, '08-7915-9999', 'jon_smith@hotmail.com'),
    (11, 'Jon', 'Smithson', 'Mr', 'GOL', 48000, '08-7915-1111', 'jon_smithson@hotmail.com')
]

# insert data into table
insert_rows(cur_b, conn_b, 'customer', customer_data)

In [54]:
# check inserted records
df = pd.read_sql('SELECT * from customer', con=conn_b)
df

,CUST_ID,FIRST_NAME,LAST_NAME,TITLE,MEMBER_TIER_CODE,MEMBER_CREDIT,CONTACT_NUM,CONTACT_EMAIL
0,1,Malcolm,Leja,Mr,SIL,11000,03-2477-9133,malcom@leja.com.au
1,2,Claudia,Gawrych,Ms,BRO,6500,02-4246-3092,claudia@gmail.com
2,3,Eliseo,Mikovec,Mr,RED,3800,02-9829-2371,emikovec@mikovec.com.au
3,4,Colene,Tolbent,Ms,GOL,25500,02-4376-1104,colene.tolbent@tolbent.net.au
4,5,Chester,Dollins,Mr,PLA,80000,02-1622-6412,chester_dollins@gmail.com
5,6,Armando,Barkley,Mr,GOL,48000,07-8371-4719,armando.barkley@yahoo.com
6,7,Tamie,Hollimon,Ms,BRO,7800,03-2444-8291,tamie@hollimon.com.au
7,8,Dylan,Chaleun,Mr,SIL,15500,08-7915-5110,dylan_chaleun@hotmail.com
8,9,John,Smith,Mr,SIL,15500,08-7915-8889,john.smith@hotmail.com
9,10,Jon,Smith,Mr,PLA,65000,08-7915-9999,jon_smith@hotmail.com


#### Booking table data
Note that the total amount values are initially 0.00. They will be updated by the trigger that computes the total amount when records on the booked_room table are inserted

In [55]:
# declare list of tuples for the data
booking_data = [
    (1, 1, 'Malcolm Leja', '03-2477-9133', 'malcom@leja.com.au', 0.00, 'Pending'),
    (2, 2, 'Claudia Gawrych', '02-4246-3092', 'claudia@gmail.com', 0.00, 'Pending'),
    (3, 3, 'Eliseo Mikovec', '02-9829-2371', 'emikovec@mikovec.com.au', 0.00, 'Pending'),
    (4, 4, 'Colene Tolbent', '02-4376-1104', 'colene.tolbent@tolbent.net.au', 0.00, 'Pending'),
    (5, 5, 'Chester Dollins', '02-1622-6412', 'chester_dollins@gmail.com', 0.00, 'Pending'),
    (6, 6, 'Armando Barkley', '07-8371-4719', 'armando.barkley@yahoo.com', 0.00, 'Pending'),
    (7, 7, 'Tamie Hollimon', '03-2444-8291', 'tamie@hollimon.com.au', 0.00, 'Pending'),
    (8, 8, 'Dylan Chaleun', '08-7915-5110', 'dylan_chaleun@hotmail.com', 0.00, 'Pending')
]

# insert data into table
insert_rows(cur_b, conn_b, 'booking', booking_data)

In [56]:
# check inserted records
df = pd.read_sql('SELECT * from booking', con=conn_b)
df

,BOOKING_ID,CUST_ID,CONTACT_NAME,CONTACT_NUM,CONTACT_EMAIL,TOTAL_AMOUNT,PAYMENT_STATUS
0,1,1,Malcolm Leja,03-2477-9133,malcom@leja.com.au,0.0,Pending
1,2,2,Claudia Gawrych,02-4246-3092,claudia@gmail.com,0.0,Pending
2,3,3,Eliseo Mikovec,02-9829-2371,emikovec@mikovec.com.au,0.0,Pending
3,4,4,Colene Tolbent,02-4376-1104,colene.tolbent@tolbent.net.au,0.0,Pending
4,5,5,Chester Dollins,02-1622-6412,chester_dollins@gmail.com,0.0,Pending
5,6,6,Armando Barkley,07-8371-4719,armando.barkley@yahoo.com,0.0,Pending
6,7,7,Tamie Hollimon,03-2444-8291,tamie@hollimon.com.au,0.0,Pending
7,8,8,Dylan Chaleun,08-7915-5110,dylan_chaleun@hotmail.com,0.0,Pending


#### Booked_Room table data

In [57]:
# declare list of tuples for the data
booked_room_data = [
    (1, 1, 101, 2, '2017-10-01', '2017-10-05'),
    (2, 1, 201, 2, '2017-11-01', '2017-11-05'),
    (2, 1, 201, 2, '2017-11-06', '2017-11-07'),
    (3, 2, 102, 2, '2017-11-10', '2017-11-15'),
    (3, 2, 103, 2, '2017-11-10', '2017-11-15'),
    (4, 2, 105, 2, '2017-11-01', '2017-11-03'),
    (4, 3, 105, 2, '2017-11-04', '2017-11-06'),
    (4, 4, 105, 2, '2017-11-07', '2017-11-10'),
    (5, 3, 206, 2, '2017-12-01', '2017-12-10'),
    (5, 4, 206, 2, '2017-12-11', '2017-12-20'),
    (5, 5, 206, 2, '2017-12-21', '2017-12-30'),
    (6, 4, 204, 2, '2017-12-20', '2017-12-26'),
    (6, 4, 205, 2, '2017-12-20', '2017-12-26'),
    (7, 1, 201, 2, '2017-12-20', '2017-12-26'),
    (7, 1, 202, 2, '2017-12-20', '2017-12-26'),
    (7, 6, 201, 2, '2017-12-20', '2017-12-26'),
    (7, 6, 202, 2, '2017-12-20', '2017-12-26'),
    (8, 6, 203, 2, '2017-10-20', '2017-10-26'),
    (8, 6, 206, 2, '2017-10-20', '2017-10-26')
]

# insert data into table
insert_rows(cur_b, conn_b, 'booked_room', booked_room_data, None, 'YYYY-MM-DD')

In [58]:
# check inserted records
df = pd.read_sql('SELECT * from booked_room', con=conn_b)
df.head(10) # show 10 rows only

,BOOKING_ID,HOTEL_ID,ROOM_NUM,GUEST_NUM,CHECKIN_DATE,CHECKOUT_DATE
0,1,1,101,2,2017-10-01,2017-10-05
1,2,1,201,2,2017-11-01,2017-11-05
2,2,1,201,2,2017-11-06,2017-11-07
3,3,2,102,2,2017-11-10,2017-11-15
4,3,2,103,2,2017-11-10,2017-11-15
5,4,2,105,2,2017-11-01,2017-11-03
6,4,3,105,2,2017-11-04,2017-11-06
7,4,4,105,2,2017-11-07,2017-11-10
8,5,3,206,2,2017-12-01,2017-12-10
9,5,4,206,2,2017-12-11,2017-12-20


After Booked_Room records are inserted, the total amounts in the Booking table were updated by a trigger, as shown below:

In [59]:
# check inserted records
df = pd.read_sql('SELECT * from booking', con=conn_b)
df

,BOOKING_ID,CUST_ID,CONTACT_NAME,CONTACT_NUM,CONTACT_EMAIL,TOTAL_AMOUNT,PAYMENT_STATUS
0,1,1,Malcolm Leja,03-2477-9133,malcom@leja.com.au,218.50,Pending
1,2,2,Claudia Gawrych,02-4246-3092,claudia@gmail.com,607.60,Pending
2,3,3,Eliseo Mikovec,02-9829-2371,emikovec@mikovec.com.au,378.10,Pending
3,4,4,Colene Tolbent,02-4376-1104,colene.tolbent@tolbent.net.au,488.06,Pending
4,5,5,Chester Dollins,02-1622-6412,chester_dollins@gmail.com,774.84,Pending
5,6,6,Armando Barkley,07-8371-4719,armando.barkley@yahoo.com,542.80,Pending
6,7,7,Tamie Hollimon,03-2444-8291,tamie@hollimon.com.au,1372.98,Pending
7,8,8,Dylan Chaleun,08-7915-5110,dylan_chaleun@hotmail.com,741.96,Pending


#### Payment table data

In [60]:
# declare list of tuples for the data
payment_data = [
    (1, '2017-09-29', 'Credit Card', 1000.00),
    (1, '2017-09-30', 'Credit Card', 500.00),
    (2, '2017-10-29', 'BPay', 1000.00),
    (2, '2017-10-30', 'Credit Card', 200.50),
    (3, '2017-11-01', 'BPay', 2500.00),
    (4, '2017-11-10', 'PayPal', 1150.00),
    (5, '2017-11-20', 'Credit Card', 1600.00),
    (5, '2017-11-25', 'PayPal', 1500.00)
]

# insert data into table
insert_rows(cur_b, conn_b, 'payment', payment_data, 
            'booking_id, payment_date, payment_method, payment_amount',
            'YYYY-MM-DD')

In [61]:
# check inserted records
df = pd.read_sql('SELECT * from payment', con=conn_b)
df

,PAYMENT_ID,BOOKING_ID,PAYMENT_DATE,PAYMENT_METHOD,PAYMENT_AMOUNT
0,1,1,2017-09-29,Credit Card,1000.0
1,2,1,2017-09-30,Credit Card,500.0
2,3,2,2017-10-29,BPay,1000.0
3,4,2,2017-10-30,Credit Card,200.5
4,5,3,2017-11-01,BPay,2500.0
5,6,4,2017-11-10,PayPal,1150.0
6,7,5,2017-11-20,Credit Card,1600.0
7,8,5,2017-11-25,PayPal,1500.0


Note that after the payment records were added, the Booking table was updated by a trigger and the payment status was set to 'Received' for those bookings with payments<br/>
See below:

In [62]:
# check inserted records
df = pd.read_sql('SELECT * from booking', con=conn_b)
df

,BOOKING_ID,CUST_ID,CONTACT_NAME,CONTACT_NUM,CONTACT_EMAIL,TOTAL_AMOUNT,PAYMENT_STATUS
0,1,1,Malcolm Leja,03-2477-9133,malcom@leja.com.au,218.50,Received
1,2,2,Claudia Gawrych,02-4246-3092,claudia@gmail.com,607.60,Received
2,3,3,Eliseo Mikovec,02-9829-2371,emikovec@mikovec.com.au,378.10,Received
3,4,4,Colene Tolbent,02-4376-1104,colene.tolbent@tolbent.net.au,488.06,Received
4,5,5,Chester Dollins,02-1622-6412,chester_dollins@gmail.com,774.84,Received
5,6,6,Armando Barkley,07-8371-4719,armando.barkley@yahoo.com,542.80,Pending
6,7,7,Tamie Hollimon,03-2444-8291,tamie@hollimon.com.au,1372.98,Pending
7,8,8,Dylan Chaleun,08-7915-5110,dylan_chaleun@hotmail.com,741.96,Pending


# Task 2- Oracle Stored Procedure (max 2 marks)
Create an Oracle Stored Procedure for updating the contact email attribute in the Hotel table. Make sure to keep the query results after you call them. Show your code works as expected.

In [63]:
cur_a.execute('''
CREATE OR REPLACE PROCEDURE updateContactEmail (
    p_contactEmail IN hotel.contact_email%TYPE,
    p_hotelName IN hotel.hotel_name%TYPE)
IS
BEGIN
    UPDATE hotel SET contact_email=p_contactEmail WHERE hotel_name=p_hotelName;
    COMMIT;
END;
''')

Let's first check out the current contact email for a specific hotel, "Hyatt Regency Sydney"

In [64]:
df = pd.read_sql('SELECT hotel_name, contact_email FROM hotel WHERE hotel_name=\'Hyatt Regency Sydney\'', con=conn_a)
df

,HOTEL_NAME,CONTACT_EMAIL
0,Hyatt Regency Sydney,reservation@hyattsyd.com


Now, let's call the stored procedure to update the contact email and set a new email address

In [65]:
cur_a.callproc('updateContactEmail', 
               ('contact@sydney.hyatt.com', 'Hyatt Regency Sydney'))

['contact@sydney.hyatt.com', 'Hyatt Regency Sydney']

Finally, let's rerun the same query to show the hotel and its updated email address

In [66]:
df = pd.read_sql('SELECT hotel_name, contact_email FROM hotel WHERE hotel_name=\'Hyatt Regency Sydney\'', con=conn_a)
df

,HOTEL_NAME,CONTACT_EMAIL
0,Hyatt Regency Sydney,contact@sydney.hyatt.com


# Task 3 – Triggers (max 3 marks)
Oracle Triggers are used to implement referential integrity across those tables that have relationships but are located across FIT5148A and FIT5148B. These triggers need to consider all the possible operations (insert, update, and delete). 

#### Room Table
A trigger will be added to the **Room** table so that during a new **insert** or an **update**, a check will be done if the hotel_id passed is valid by checking if it exists in the **Hotel** table.

In [67]:
# trigger to check if hotel_id set in the room table is valid by checking if it exists in the hotel table
# this is called for both the INSERT and UPDATE actions on the room table
cur_b.execute('''
CREATE OR REPLACE TRIGGER room_trigger_biu
BEFORE INSERT OR UPDATE ON room
FOR EACH ROW
DECLARE 
    hotelIdCount INTEGER;
BEGIN
    -- check if hotel_id argument exists in the hotel table
    SELECT COUNT(1) INTO hotelIdCount FROM hotel@FIT5148A WHERE hotel_id=:new.hotel_id;
    -- raise an error if hotel_id does not exist in the hotel table
    IF(hotelIdCount < 1) THEN
        RAISE_APPLICATION_ERROR(-20000, 'Integrity constraint violated: Invalid Hotel Id');
    END IF;    
END;
''')
conn_b.commit()

#### Hotel Table
Two triggers will be added on the **Hotel** table.<br/>
When an **update** or **delete** is done on the Hotel table, a check will be done if the hotel_id being updated is used in the **Room** table.

In [68]:
# trigger to check if a record in the room table refers to a hotel_id for a record being deleted in the hotel table
cur_a.execute('''
CREATE OR REPLACE TRIGGER hotel_trigger_bd
BEFORE DELETE ON hotel
FOR EACH ROW
DECLARE 
    hotelIdCount INTEGER;
BEGIN
    -- check if hotel_id for record being deleted is used in the room table
    SELECT COUNT(1) INTO hotelIdCount FROM room@FIT5148B WHERE hotel_id=:old.hotel_id;
    -- raise an error if hotel_id is used.
    IF(hotelIdCount >= 1) THEN
        RAISE_APPLICATION_ERROR(-20000, 'Integrity constraint violated: Hotel Id being deleted is used.');
    END IF;    
END;
''')
conn_a.commit()

In [69]:
# trigger to check if a record in the room table refers to a hotel_id being replaced in the hotel table
cur_a.execute('''
CREATE OR REPLACE TRIGGER hotel_trigger_bu
BEFORE UPDATE ON hotel
FOR EACH ROW
DECLARE 
    hotelIdCount INTEGER;
BEGIN
    IF (:old.hotel_id != :new.hotel_id) THEN
        -- check if hotel_id for record being deleted is used in the room table
        SELECT COUNT(1) INTO hotelIdCount FROM room@FIT5148B WHERE hotel_id=:old.hotel_id;
        -- raise an error if hotel_id is used.
        IF(hotelIdCount >= 1) THEN
            RAISE_APPLICATION_ERROR(-20000, 'Integrity constraint violated: Hotel Id being modified is used.');
        END IF;
    END IF;
END;
''')
conn_a.commit()

*Sample test cases:*<br/>
Case 1. Inserting a new record into the room table where the hotel_id is invalid ---> a custom error is raised by the trigger with message "**Integrity constraint violated: Invalid Hotel Id**"

In [70]:
try:
    cur_b.execute('''INSERT INTO room (hotel_id, room_num, room_type_code, occupancy, rate, description) VALUES (99, 106, 'K', 3, 230.00, 'King room, smoking')''')
except Exception as e:
    print('Exception: ', e)

Exception:  ORA-20000: Integrity constraint violated: Invalid Hotel Id
ORA-06512: at "S28037537.ROOM_TRIGGER_BIU", line 8
ORA-04088: error during execution of trigger 'S28037537.ROOM_TRIGGER_BIU'


Case 2: Updating an existing record in the room table with an invalid hotel_id --> a custom error is raised by the trigger with message "**Integrity constraint violated: Invalid Hotel Id**"

In [71]:
try:
    cur_b.execute('UPDATE room set hotel_id=99 WHERE hotel_id=1')
except Exception as e:
    print('Exception: ', e)

Exception:  ORA-20000: Integrity constraint violated: Invalid Hotel Id
ORA-06512: at "S28037537.ROOM_TRIGGER_BIU", line 8
ORA-04088: error during execution of trigger 'S28037537.ROOM_TRIGGER_BIU'


Case 3: Changing the hotel_id in the hotel table when records in the room table refer to it ---> a custom error is raised by the trigger with message "**Integrity constraint violated: Hotel Id being modified is used.**"

In [72]:
try:
    cur_a.execute('UPDATE hotel set hotel_id=99 WHERE hotel_id=1')
except Exception as e:
    print('Exception: ', e)

Exception:  ORA-20000: Integrity constraint violated: Hotel Id being modified is used.
ORA-06512: at "S28037537.HOTEL_TRIGGER_BU", line 9
ORA-04088: error during execution of trigger 'S28037537.HOTEL_TRIGGER_BU'


Case 4: Deleting a hotel record from the hotel table that is referenced by atleast one record in the room table ---> a custom error is raised by the trigger with message "**Integrity constraint violated: Hotel Id being deleted is used.**"

In [73]:
try:
    cur_a.execute('DELETE FROM hotel WHERE hotel_id=1')
except Exception as e:
    print('Exception: ', e)

Exception:  ORA-20000: Integrity constraint violated: Hotel Id being deleted is used.
ORA-06512: at "S28037537.HOTEL_TRIGGER_BD", line 8
ORA-04088: error during execution of trigger 'S28037537.HOTEL_TRIGGER_BD'


# Task 4- Fragmentation (max 10 marks)


## Horizontal 
Discuss your assumptions, user application and design here.
Please provide the lists of **simple predicates** and **minterm predicates** that you used.

A candidate relation/table for Horizontal Fragmentation is the **Customer** table.<br/>
Sample rows displayed below:

In [74]:
df = pd.read_sql('SELECT * FROM customer ORDER BY title, member_credit', con=conn_b)
df

,CUST_ID,FIRST_NAME,LAST_NAME,TITLE,MEMBER_TIER_CODE,MEMBER_CREDIT,CONTACT_NUM,CONTACT_EMAIL
0,3,Eliseo,Mikovec,Mr,RED,6300,02-9829-2371,emikovec@mikovec.com.au
1,1,Malcolm,Leja,Mr,SIL,12500,03-2477-9133,malcom@leja.com.au
2,9,John,Smith,Mr,SIL,15500,08-7915-8889,john.smith@hotmail.com
3,8,Dylan,Chaleun,Mr,SIL,15500,08-7915-5110,dylan_chaleun@hotmail.com
4,6,Armando,Barkley,Mr,GOL,48000,07-8371-4719,armando.barkley@yahoo.com
5,11,Jon,Smithson,Mr,GOL,48000,08-7915-1111,jon_smithson@hotmail.com
6,10,Jon,Smith,Mr,PLA,65000,08-7915-9999,jon_smith@hotmail.com
7,5,Chester,Dollins,Mr,PLA,83100,02-1622-6412,chester_dollins@gmail.com
8,2,Claudia,Gawrych,Ms,BRO,7701,02-4246-3092,claudia@gmail.com
9,7,Tamie,Hollimon,Ms,BRO,7800,03-2444-8291,tamie@hollimon.com.au


#### Simple Predicates
##### Application 1
Suppose that because of different targeted marketing initiatives, one application differentiates between male and female customers.<br/>
A proxy for the gender is the **title** column. (Mr = male, Ms = female)<br/>
<br/>
For this application, the simple predicates that would be used are the following:<br>
<br/>
$p_1$: title = 'Mr'<br/>
$p_2$: title = 'Ms'

##### Application 2
The second application is interested in membership credits.<br/>
Those with credits greater than or equal to 25000 ('Gold' or higher membership) are processed at one site, while those with less than 25000 are managed at another.<br/>
<br/>
For this application, the simple predicate are:<br/>
<br/>
$p_3:$ member_credit >= 25000<br/>
$p_4:$ member_credit < 25000

#### Minterm Predicates
The following are minterm predicates that can be defined based on the simple predicates:<br/>
<br/>
$m_1$: title = 'Mr' $\wedge$ member_credit >= 25000 <br/>
$m_2$: title = 'Mr' $\wedge$ NOT(member_credit >= 25000) <br/>
$m_3$: title = 'Ms' $\wedge$ member_credit >= 25000 <br/>
$m_4$: title = 'Ms' $\wedge$ NOT(member_credit >= 25000) <br/>
$m_5$: NOT(title = 'Mr') $\wedge$ member_credit >= 25000 <br/>
$m_6$: NOT(title = 'Mr') $\wedge$ NOT(member_credit >= 25000) <br/>
$m_7$: NOT(title = 'Ms') $\wedge$ member_credit >= 25000 <br/>
$m_8$: NOT(title = 'Ms') $\wedge$ NOT(member_credit >= 25000) <br/>
$m_9$: title = 'Mr' $\wedge$ member_credit < 25000 <br/>
$m_{10}$: title = 'Mr' $\wedge$ NOT(member_credit < 25000) <br/>
$m_{11}$: title = 'Ms' $\wedge$ member_credit < 25000 <br/>
$m_{12}$: title = 'Ms' $\wedge$ NOT(member_credit < 25000) <br/>
$m_{13}$: NOT(title = 'Mr') $\wedge$ member_credit < 25000 <br/>
$m_{14}$: NOT(title = 'Mr') $\wedge$ NOT(member_credit < 25000) <br/>
$m_{15}$: NOT(title = 'Ms') $\wedge$ member_credit < 25000 <br/>
$m_{16}$: NOT(title = 'Ms') $\wedge$ NOT(member_credit < 25000) <br/>
<br/>
After eliminating duplicates, we are left with the following minterm predicates:<br/>
<br/>
$m_1$: title = 'Mr' $\wedge$ member_credit >= 25000 <br/>
$m_2$: title = 'Mr' $\wedge$ member_credit < 25000 <br/>
$m_3$: title = 'Ms' $\wedge$ member_credit >= 25000 <br/>
$m_4$: title = 'Ms' $\wedge$ member_credit < 25000 <br/>

### Horizontal - Populate Tables
For horizontal fragmentation, we'll distribute the customer table into two fragments, separating based on the title value.

In [75]:
# drop tables if they exist.
drop_table(cur_a, 'customer_male')
drop_table(cur_b, 'customer_female')

In [76]:
# We put the male customers on FIT5148A
cur_a.execute ('''
CREATE TABLE customer_male 
AS (SELECT * from customer@FIT5148B WHERE title='Mr')''')
conn_a.commit()

In [77]:
# check out records
df = pd.read_sql('SELECT * FROM customer_male', con=conn_a)
df

,CUST_ID,FIRST_NAME,LAST_NAME,TITLE,MEMBER_TIER_CODE,MEMBER_CREDIT,CONTACT_NUM,CONTACT_EMAIL
0,1,Malcolm,Leja,Mr,SIL,12500,03-2477-9133,malcom@leja.com.au
1,3,Eliseo,Mikovec,Mr,RED,6300,02-9829-2371,emikovec@mikovec.com.au
2,5,Chester,Dollins,Mr,PLA,83100,02-1622-6412,chester_dollins@gmail.com
3,6,Armando,Barkley,Mr,GOL,48000,07-8371-4719,armando.barkley@yahoo.com
4,8,Dylan,Chaleun,Mr,SIL,15500,08-7915-5110,dylan_chaleun@hotmail.com
5,9,John,Smith,Mr,SIL,15500,08-7915-8889,john.smith@hotmail.com
6,10,Jon,Smith,Mr,PLA,65000,08-7915-9999,jon_smith@hotmail.com
7,11,Jon,Smithson,Mr,GOL,48000,08-7915-1111,jon_smithson@hotmail.com


In [78]:
# We put the female customers on FIT5148B
cur_b.execute ('''
CREATE TABLE customer_female 
AS (SELECT * from customer WHERE title='Ms')''')
conn_b.commit()

In [79]:
# check out records
df = pd.read_sql('SELECT * FROM customer_female', con=conn_b)
df

,CUST_ID,FIRST_NAME,LAST_NAME,TITLE,MEMBER_TIER_CODE,MEMBER_CREDIT,CONTACT_NUM,CONTACT_EMAIL
0,2,Claudia,Gawrych,Ms,BRO,7701,02-4246-3092,claudia@gmail.com
1,4,Colene,Tolbent,Ms,GOL,26650,02-4376-1104,colene.tolbent@tolbent.net.au
2,7,Tamie,Hollimon,Ms,BRO,7800,03-2444-8291,tamie@hollimon.com.au


## Vertical 
Discuss your assumptions, user application and design here. Please provide four queries, Usage Matrix, Access Frequency Matrix for two sites of **FIT5148A** and **FIT5148B**, and **Attribute Affinity Matrix** of all the attributes.

For Vertical Fragmentation, let us consider the **Booking** table.<br/>
Sample rows below:

In [80]:
df = pd.read_sql('SELECT * FROM booking', con=conn_b)
df.head(15)

,BOOKING_ID,CUST_ID,CONTACT_NAME,CONTACT_NUM,CONTACT_EMAIL,TOTAL_AMOUNT,PAYMENT_STATUS
0,1,1,Malcolm Leja,03-2477-9133,malcom@leja.com.au,218.50,Received
1,2,2,Claudia Gawrych,02-4246-3092,claudia@gmail.com,607.60,Received
2,3,3,Eliseo Mikovec,02-9829-2371,emikovec@mikovec.com.au,378.10,Received
3,4,4,Colene Tolbent,02-4376-1104,colene.tolbent@tolbent.net.au,488.06,Received
4,5,5,Chester Dollins,02-1622-6412,chester_dollins@gmail.com,774.84,Received
5,6,6,Armando Barkley,07-8371-4719,armando.barkley@yahoo.com,542.80,Pending
6,7,7,Tamie Hollimon,03-2444-8291,tamie@hollimon.com.au,1372.98,Pending
7,8,8,Dylan Chaleun,08-7915-5110,dylan_chaleun@hotmail.com,741.96,Pending


The following are the applications and SQL queries run against the Booking table:<br/>
<br/>
$q_1$: Find the contact and contact number for a particular booking id:<br/>
<br/>
SELECT contact_name, contact_num<br/>
FROM booking<br/>
WHERE booking_id = *value*<br/>
<br/>
$q_2$: Find the total revenue (total amount in Received status):<br/>
<br/>
SELECT SUM(total_amount)<br/>
FROM booking<br/>
WHERE payment_status='Received'<br/>
<br/>
$q_3$: Find contact info for bookings in Pending status<br/>
<br/>
SELECT contact_name, contact_num<br/>
FROM booking<br/>
WHERE payment_status='Pending'<br/>
<br/>
$q_4$: Find bookings where total amount is above a threshold value<br/>
<br/>
SELECT \*<br/>
FROM booking<br/>
WHERE total_amount > *value*<br/>


The attributes are:<br/>
$A_1$ = contact_name<br/>
$A_2$ = contact_num<br/>
$A_3$ = total_amount<br/>
$A_4$ = payment_status.<br/>
<br/>
The **Attribute usage values** are defined in matrix form<br/>:
![attrib_usage_values](./attrib_usage_values.png)

**Access Frequencies** - let's assume that the access frequencies for 3 sites are:<br/>
<br/>
$acc_1(q_1)$ = 10 $acc_2(q_1)$ = 20 $acc_3(q_1)$ = 10<br/>
$acc_1(q_2)$ = 15 $acc_2(q_2)$ = 15 $acc_3(q_2)$ = 15<br/>
$acc_1(q_3)$ = 5 $acc_2(q_3)$ = 0 $acc_3(q_3)$ = 20<br/>
$acc_1(q_4)$ = 1 $acc_2(q_4)$ = 0 $acc_3(q_4)$ = 0<br/>
<br/>
In matrix form:<br/>
note: S1, S2, S3 represent the 3 sites<br/>
<br/>
![access_frequencies](./access_frequencies.png)

**Attribute Affinity Matrix (AA)** calculation:<br/>
<br/>
Given the Attribute usage values and Access frequencies, the Attribute Affinity Matrix can be calculated as:<br/>
<br/>
![attrib_affinity_matrix](./attrib_affinity_matrix.png)

### Vertical - Populate Tables 

We assume that A1 (contact_name) and A2 (contact_num) (and the rest of contact-related info) have more affinity with each other<br/>
and that A3 (total_amount) and A4 (payment_status) have more affinity with each other.<br/>
So we will fragment the table vertically based on these assumptions and distribute them between two locations.

In [81]:
# drop tables if they exist.
drop_table(cur_a, 'booking_contacts')
drop_table(cur_b, 'booking_amounts')

In [82]:
# We put the contact related data on FIT5148A
cur_a.execute ('''
CREATE TABLE booking_contacts
AS (SELECT booking_id, cust_id, contact_name, contact_num, contact_email from booking@FIT5148B)''')
conn_a.commit()

In [83]:
# check out records
df = pd.read_sql('SELECT * FROM booking_contacts', con=conn_a)
df

,BOOKING_ID,CUST_ID,CONTACT_NAME,CONTACT_NUM,CONTACT_EMAIL
0,1,1,Malcolm Leja,03-2477-9133,malcom@leja.com.au
1,2,2,Claudia Gawrych,02-4246-3092,claudia@gmail.com
2,3,3,Eliseo Mikovec,02-9829-2371,emikovec@mikovec.com.au
3,4,4,Colene Tolbent,02-4376-1104,colene.tolbent@tolbent.net.au
4,5,5,Chester Dollins,02-1622-6412,chester_dollins@gmail.com
5,6,6,Armando Barkley,07-8371-4719,armando.barkley@yahoo.com
6,7,7,Tamie Hollimon,03-2444-8291,tamie@hollimon.com.au
7,8,8,Dylan Chaleun,08-7915-5110,dylan_chaleun@hotmail.com


In [84]:
# We put the payment amount related data on FIT5148B
cur_b.execute ('''
CREATE TABLE booking_amounts
AS (SELECT booking_id, total_amount, payment_status from booking)''')
conn_b.commit()

In [85]:
# check out records
df = pd.read_sql('SELECT * FROM booking_amounts', con=conn_b)
df

,BOOKING_ID,TOTAL_AMOUNT,PAYMENT_STATUS
0,1,218.50,Received
1,2,607.60,Received
2,3,378.10,Received
3,4,488.06,Received
4,5,774.84,Received
5,6,542.80,Pending
6,7,1372.98,Pending
7,8,741.96,Pending


**Note 1:** You should implement both types of fragments on the database such that they are populated with original tables’ data. Use different and meaningful names for each fragment. 

**Note 2:** Two implemented fragments CANNOT be in the same location. 

**Note 3:** Provide your assumptions, user applications and queries, lists of predicates and minterm predicates, specified matrices and your SQL code to create fragments on two locations.


# Task 5 – ETL Demonstration (max 7 marks)

## 5.1 
Search hotels (Hotel table) by city. Your code should work properly when the queried city is not listed (0.5 mark)

In [86]:
def get_hotels_by_city(city):
    cur_a.execute('SELECT hotel_name FROM hotel WHERE city=\'{}\''.format(city)) 
    rows = cur_a.fetchall()
    if (len(rows) > 0):
        print('Hotels for {}:'.format(city))
        i = 0
        for row in rows:
            i += 1
            print('{}. {}'.format(i, row[0]))
    else:
        print('No hotel found for {}'.format(city))

Sample of a search for hotels for a city in the database:

In [87]:
get_hotels_by_city('Sydney')

Hotels for Sydney:
1. ParkRoyal Darling Harbour
2. Hyatt Regency Sydney


Sample of a search for hotels for a city NOT in the database:

In [88]:
get_hotels_by_city('Darwin')

No hotel found for Darwin


## 5.2 
Update (Room table) the type of a room. Your code should work properly when the new room type is invalid. (0.5 mark)

In [89]:
def update_room_type(hotel_name, room_num, new_room_type):
    # get the room_type_code given the room type name
    cur_b.execute('SELECT room_type_code FROM room_type where room_type_name=\'{}\''.format(new_room_type))
    room_type_row = cur_b.fetchone()
    if (room_type_row):
        # get the hotel id given the hotel name
        cur_a.execute('SELECT hotel_id FROM hotel where hotel_name=\'{}\''.format(hotel_name))
        hotel_row = cur_a.fetchone()
        if (hotel_row):
            # update the room table with the new room type code
            room_type_code = room_type_row[0]
            hotel_id = hotel_row[0]
            cur_b.execute('UPDATE room SET room_type_code=\'{}\' WHERE hotel_id={} AND room_num={}'.format(
                room_type_code, hotel_id, room_num))
            conn_b.commit()
        else:
            print('Hotel name passed is invalid.')
    else:
        print('Room type passed is invalid.')

Here's a valid sample which updates the room type of a room in Hilton Brisbane hotel.<br/>
First, show the current room type:

In [90]:
# declare a function to show room info
def show_room_info(hotel_name, room_num):
    df = pd.read_sql('''
        SELECT h.hotel_name, r.room_num, rt.room_type_name 
        FROM hotel@FIT5148A h 
            INNER JOIN room r ON h.hotel_id=r.hotel_id 
            INNER JOIN room_type rt ON r.room_type_code=rt.room_type_code
        WHERE h.hotel_name=\'{}\' AND r.room_num={}'''.format(hotel_name, room_num), 
                     con=conn_b)
    print(df)

For this sample, we will work on 'Hilton Brisbane', room # 105.

In [91]:
show_room_info('Hilton Brisbane', 105)

        HOTEL_NAME  ROOM_NUM ROOM_TYPE_NAME
0  Hilton Brisbane       105    Single room


Next, run the function to update the room type

In [92]:
update_room_type('Hilton Brisbane', 105, 'Queen room')

Then, display the updated room details which show the updated room type:

In [93]:
show_room_info('Hilton Brisbane', 105)

        HOTEL_NAME  ROOM_NUM ROOM_TYPE_NAME
0  Hilton Brisbane       105     Queen room


Here is a sample case where the passed room type is invalid (because the room type argument passed is not defined in the **room_type** table)

In [94]:
update_room_type('Hilton Brisbane', 105, 'Dining room')

Room type passed is invalid.


Run the query again to show that the room type was not updated

In [95]:
show_room_info('Hilton Brisbane', 105)

        HOTEL_NAME  ROOM_NUM ROOM_TYPE_NAME
0  Hilton Brisbane       105     Queen room


## 5.3 
Search eligible membership by available credits (i.e. tier credits <= available credits) and return results sorted based guest names. Show your code works properly when there are guests with similar surname. (1 mark)

In [96]:
def search_eligible_members(tier):
    cur_b.execute('''SELECT first_name, last_name 
        FROM customer 
        WHERE member_credit > 
            (SELECT tier_credit FROM membership WHERE member_tier_name=\'{}\')'''.format(tier))
    rows = cur_b.fetchall()
    if (len(rows) > 0):
        print('Eligible customers for {} membership:'.format(tier))
        i = 0
        for row in rows:
            i += 1
            print('{}. {} {}'.format(i, row[0], row[1]))
    else:
        print('No eligible customers for {} membership'.format(city))    

To illustrate, we'll find out who are eligible members for **Platinum Elite**.<br/>
First, we'll check out the minimum credits for this tier.

In [97]:
df = pd.read_sql('SELECT member_tier_name, tier_credit FROM membership WHERE member_tier_name=\'Platinum Elite\'', con=conn_b)
df

,MEMBER_TIER_NAME,TIER_CREDIT
0,Platinum Elite,50000


Next, we run the function to get eligible customers:

In [98]:
search_eligible_members('Platinum Elite')

Eligible customers for Platinum Elite membership:
1. Chester Dollins
2. Jon Smith


The following query illustrates that the code above works for customers with similar surnames. <br/>
There are three customers with similar names but only one of them was eligible.

In [99]:
df = pd.read_sql('SELECT first_name, last_name, member_credit FROM customer WHERE last_name LIKE \'Smith%\'', con=conn_b)
df

,FIRST_NAME,LAST_NAME,MEMBER_CREDIT
0,John,Smith,15500
1,Jon,Smith,65000
2,Jon,Smithson,48000


## 5.4 
Provide a search query that searches available room by check-in and check-out date, room type, occupancy, and rate range. Only show 10 items. Also make sure the code provide appropriate output if it cannot not find any available room.  (2 marks)

For this task, we will create two Views.<br/>
**all_rooms** - view containing all hotel rooms that combine data from the **hotel** and **room** tables.<br/>
**confirmed_booked_rooms** - view containing booked rooms that have not been *Cancelled*

In [100]:
cur_b.execute('''
CREATE OR REPLACE VIEW all_rooms AS
    SELECT h.hotel_id, h.hotel_name, r.room_num, r.room_type_code, r.occupancy, r.rate 
    FROM room r INNER JOIN hotel@FIT5148A h ON h.hotel_id=r.hotel_id''')

In [101]:
cur_b.execute('''
CREATE OR REPLACE VIEW confirmed_booked_rooms AS
    SELECT * from booked_room 
    WHERE booking_id IN (
        SELECT booking_id FROM booking WHERE payment_status != 'Cancelled')''')

In [102]:
conn_b.commit()

In [103]:
# function for searching for available rooms
def find_available_room(checkin_date, checkout_date, room_type_code, occupancy, min_rate, max_rate):
    sql = '''
        SELECT a.hotel_id, a.hotel_name, a.room_num, a.room_type_code, a.occupancy 
        FROM all_rooms a
        WHERE a.occupancy={0}
        AND a.room_type_code='{1}'
        AND a.rate BETWEEN {2} and {3}
        AND NOT EXISTS 
            (SELECT c.* FROM confirmed_booked_rooms c 
             WHERE c.hotel_id=a.hotel_id AND c.room_num=a.room_num
             AND ((c.checkin_date <= TO_DATE('{4}', 'YYYY-MM-DD') AND c.checkout_date >= TO_DATE('{4}', 'YYYY-MM-DD'))
                  OR
                  (c.checkin_date <= TO_DATE('{5}', 'YYYY-MM-DD') AND c.checkout_date >= TO_DATE('{5}', 'YYYY-MM-DD')))
            )
        ORDER BY a.hotel_id, a.room_num
    '''.format(occupancy, room_type_code, min_rate, max_rate, checkin_date, checkout_date)
    
    df = pd.read_sql(sql, con=conn_b)
    if (not df.empty):
        print('Available rooms:\n')
        print(df.head(10))  # show only 10 rows
    else:
        print('No room available for specified criteria.')

Here are sample cases for searches with different criteria:

In [104]:
# King (K), occupancy=2, rate from $200 to $300 for specified date
find_available_room('2017-11-01', '2017-11-10', 'K', 2, 200, 300)

Available rooms:

   HOTEL_ID               HOTEL_NAME  ROOM_NUM ROOM_TYPE_CODE  OCCUPANCY
0         3  InterContinental Rialto       201              K          2
1         3  InterContinental Rialto       202              K          2
2         3  InterContinental Rialto       203              K          2
3         3  InterContinental Rialto       204              K          2
4         3  InterContinental Rialto       205              K          2
5         3  InterContinental Rialto       206              K          2


In [105]:
# Studio (STU), occupancy=2, rate from $200 to $300 for specified date
find_available_room('2017-12-20', '2017-12-26', 'STU', 2, 200, 300)

Available rooms:

   HOTEL_ID        HOTEL_NAME  ROOM_NUM ROOM_TYPE_CODE  OCCUPANCY
0         4  Vibe Savoy Hotel       201            STU          2
1         4  Vibe Savoy Hotel       202            STU          2
2         4  Vibe Savoy Hotel       203            STU          2


In [106]:
# We will have different options if we increased the rate range
# Studio (STU), occupancy=2, rate from $300 to $400 for specified date
find_available_room('2017-12-20', '2017-12-26', 'STU', 2, 300, 400)

Available rooms:

   HOTEL_ID                 HOTEL_NAME  ROOM_NUM ROOM_TYPE_CODE  OCCUPANCY
0         1  ParkRoyal Darling Harbour       203            STU          2
1         1  ParkRoyal Darling Harbour       204            STU          2
2         1  ParkRoyal Darling Harbour       205            STU          2
3         1  ParkRoyal Darling Harbour       206            STU          2
4         6   Novotel Surfers Paradise       203            STU          2
5         6   Novotel Surfers Paradise       204            STU          2
6         6   Novotel Surfers Paradise       205            STU          2
7         6   Novotel Surfers Paradise       206            STU          2


In [107]:
# And if we avoid the dates near Christmas, even more rooms become available 
# Studio (STU), occupancy=2, rate from $300 to $400 for specified date
find_available_room('2017-12-01', '2017-12-06', 'STU', 2, 300, 400)

Available rooms:

   HOTEL_ID                 HOTEL_NAME  ROOM_NUM ROOM_TYPE_CODE  OCCUPANCY
0         1  ParkRoyal Darling Harbour       201            STU          2
1         1  ParkRoyal Darling Harbour       202            STU          2
2         1  ParkRoyal Darling Harbour       203            STU          2
3         1  ParkRoyal Darling Harbour       204            STU          2
4         1  ParkRoyal Darling Harbour       205            STU          2
5         1  ParkRoyal Darling Harbour       206            STU          2
6         6   Novotel Surfers Paradise       201            STU          2
7         6   Novotel Surfers Paradise       202            STU          2
8         6   Novotel Surfers Paradise       203            STU          2
9         6   Novotel Surfers Paradise       204            STU          2


## 5.5 
Make a payment, show the updates, cancell it, and show the final status. After a payment is made/cancelled, membership credits should be awarded to/deducted from the user accordingly. Payment status of booking should be updated correspondingly as well.  (2 marks)

This requirement is already taken care of by the configured trigger on the **payment** table.<br/>
<br/>
When a payment is made, membership credits are added to the customer and the payment status of the booking is set to 'Received'
<br/>
When a payment is cancelled, membership credits are deducted from the customer credit and payment status of the booking is set to 'Cancelled'.<br/>
<br/>
Following is an illustration:<br/>
We will work on booking_id = 8 for customer "Dylan Chaleun" (cust_id = 8)

In [108]:
# current state showing booking status
df = pd.read_sql('SELECT booking_id, cust_id, total_amount, payment_status FROM booking WHERE booking_id=8', con=conn_b)
df

,BOOKING_ID,CUST_ID,TOTAL_AMOUNT,PAYMENT_STATUS
0,8,8,741.96,Pending


In [109]:
# current state showing customer credits
df = pd.read_sql('SELECT cust_id, first_name, last_name, member_credit FROM customer WHERE cust_id=8', con=conn_b)
df

,CUST_ID,FIRST_NAME,LAST_NAME,MEMBER_CREDIT
0,8,Dylan,Chaleun,15500


** 5.5.1. Payment **<br/>
Now, make a payment by inserting a record into the **payment** table for the booking id

In [110]:
cur_b.execute('''
INSERT INTO payment (booking_id, payment_date, payment_method, payment_amount)
VALUES (8, TO_DATE('2017-09-10', 'YYYY-MM-DD'), 'BPay', 741.96)
''')
conn_b.commit()

Now check the booking and customer records
- Payment_Status updated to 'Received'
- member_credits has increased

In [111]:
# current state showing booking status
df = pd.read_sql('SELECT booking_id, cust_id, total_amount, payment_status FROM booking WHERE booking_id=8', con=conn_b)
df

,BOOKING_ID,CUST_ID,TOTAL_AMOUNT,PAYMENT_STATUS
0,8,8,741.96,Received


In [112]:
# current state showing customer credits
df = pd.read_sql('SELECT cust_id, first_name, last_name, member_credit FROM customer WHERE cust_id=8', con=conn_b)
df

,CUST_ID,FIRST_NAME,LAST_NAME,MEMBER_CREDIT
0,8,Dylan,Chaleun,16242


** 5.5.2. Cancel Payment **<br/>
Now, make cancel the payment by inserting a debit (negative amount) into the **payment** table for the booking id

In [113]:
cur_b.execute('''
INSERT INTO payment (booking_id, payment_date, payment_method, payment_amount)
VALUES (8, TO_DATE('2017-09-10', 'YYYY-MM-DD'), 'BPay', -741.96)
''')
conn_b.commit()

Now check the booking and customer records
- Payment_Status updated to 'Cancelled'
- member_credits have decreased

In [114]:
# current state showing booking status
df = pd.read_sql('SELECT booking_id, cust_id, total_amount, payment_status FROM booking WHERE booking_id=8', con=conn_b)
df

,BOOKING_ID,CUST_ID,TOTAL_AMOUNT,PAYMENT_STATUS
0,8,8,741.96,Cancelled


In [115]:
# current state showing customer credits
df = pd.read_sql('SELECT cust_id, first_name, last_name, member_credit FROM customer WHERE cust_id=8', con=conn_b)
df

,CUST_ID,FIRST_NAME,LAST_NAME,MEMBER_CREDIT
0,8,Dylan,Chaleun,15500


## 5.6 
Find the name and available credit  of the guest who booked the most number of rooms. (1 mark)
Provide Python and SQL codes and results.

This task is interpreted as finding the customer who made the most bookings.<br/>
As such, it is not enough to find the most number of rooms booked. We need to find out the one who made the most number of rooms booked for the most number of days.<br/>
<br/>
To help us with this, we can build a view with a derived column with the number of booked days, calculated by getting the difference in days between the checkout and checkin dates.

In [116]:
cur_b.execute('''
CREATE OR REPLACE VIEW confirmed_booked_days AS
    SELECT b.cust_id, br.hotel_id, br.room_num, (br.checkout_date - br.checkin_date) as booked_days
    FROM booking b INNER JOIN booked_room br ON b.booking_id=br.booking_id 
    WHERE b.payment_status != 'Cancelled'
''')
conn_b.commit()

Then we can select records from this view, group them by customer id and summing up the calculated number of booked days.<br/>
We sort the result by total number of booked days in descending order.

In [117]:
df = pd.read_sql('''
SELECT cust_id, sum(booked_days) AS sum_booked_days 
FROM confirmed_booked_days
GROUP by cust_id
ORDER by sum_booked_days desc
''', con=conn_b)
df

,CUST_ID,SUM_BOOKED_DAYS
0,5,27
1,7,24
2,6,12
3,3,10
4,4,7
5,2,5
6,1,4


Finally, we combine a few SQL statements together to get the top customer who made the most bookings and his available credits.

In [118]:
df = pd.read_sql('''
    SELECT first_name, last_name, member_credit
    FROM customer
    WHERE cust_id IN
        (SELECT cust_id FROM
            (SELECT cust_id, sum(booked_days) AS sum_booked_days 
            FROM confirmed_booked_days
            GROUP by cust_id
            ORDER by sum_booked_days desc)
        WHERE ROWNUM=1)
''', con=conn_b)
df

,FIRST_NAME,LAST_NAME,MEMBER_CREDIT
0,Chester,Dollins,83100


**Notes:**
* You need to prepopulate your tables such that you can demonstrate different scenarios when needed (eg, Task 5.3).
* Make sure you provide your code, comments and the results of queries.
* DO NOT create a database connection in each frame. There is a mark deduction (up to 3 marks) if you make multiple connections to the same database concurrently and if you do not CLOSE (2 marks) the connections properly.


In [119]:
# close cursors and connections
cur_a.close()
conn_a.close()

cur_b.close()
conn_b.close()

Final reminders:
* Providing SQL code, ER diagram, Python code and results in the Task order with numbered headings as specified in each requirement
* PDF file of the last version of the Jupyter Notebook after all tasks are completed and executed (including all the results). You only need to perform **“Export as PDF”**.
* **Submisssion delay**: 5% penalty per day including weekends. Submissions with one week delay will not be assessed
* **Extensions**: You must submit a special consideration form and provide valid documentation such as a medical certificate prior to the submission deadline (NOT after). Please contact [MonashOnline.StudentSuccess](mailto:MonashOnline.StudentSuccess@monash.edu)
* All the necessary files should be uploaded to Moodle as a zip file. Please use the following naming convention: ```FIT5148-A1-[StudentID].zip```. There is an effective **mark deduction** for missing documents, use of non-portable addresses, and non-standard file names.